|  |  |  |
| ---: | :--- | :--- |
| Курс:| Предобработка данных | 01 |
| Срок обучения на момент сдачи: | 2 недели |


# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Содержание/план

1  [Вводные данные и постановка задачи](#1)

2  [Импорт библиотек, загрузка данных, изучение данных](#2)

3  [Предобработка данных](#3)

* 3.1  [Устранение ошибок](#31)
* 3.2  [Обработка пропусков](#32)
* 3.3  [Замена типа данных](#33)
* 3.4  [Обработка дубликатов](#34)

4  [Лемматизация целей кредита](#4)

5  [Категоризация данных](#5)

6  [Ответы на вопросы задачи](#6)

7  [Общий вывод](#7)

<a name="1"></a>
## Вводные данные и постановка задачи

Описание данных:
*    children — количество детей в семье
*    days_employed — общий трудовой стаж в днях
*    dob_years — возраст клиента в годах
*    education — уровень образования клиента
*    education_id — идентификатор уровня образования
*    family_status — семейное положение
*    family_status_id — идентификатор семейного положения
*    gender — пол клиента
*    income_type — тип занятости
*    debt — имел ли задолженность по возврату кредитов
*    total_income — ежемесячный доход
*    purpose — цель получения кредита

Ответьте на вопросы:

*   Есть ли зависимость между наличием детей и возвратом кредита в срок?
*   Есть ли зависимость между семейным положением и возвратом кредита в срок?
*   Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
*   Как разные цели кредита влияют на его возврат в срок?

<a name="2"></a>
## Импорт библиотек, загрузка данных, изучение данных 

In [1]:
#pip install jupyter_contrib_nbextensions #для включения навигации
#pip install autopep8 #для включения автопроверки соответствия кода pep8

In [2]:
import pandas as pd
from IPython.display import display
from pymystem3 import Mystem

In [3]:
data=pd.read_csv('/datasets/data.csv')

In [4]:
data.info()

display(data.head(5))

print(data.describe())
print()

for col in ['children', 'dob_years', 'education', 'family_status', 'gender', 'income_type', 'purpose']:
    print(col)
    print(data[col].value_counts())
    print()

print('количество отрицательных значений days_employed:', len(data[data['days_employed']<0]))
print()

print('количество нулевых значений возраста:', data[data['dob_years'] == 0]['dob_years'].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


           children  days_employed     dob_years  education_id  \
count  21525.000000   19351.000000  21525.000000  21525.000000   
mean       0.538908   63046.497661     43.293380      0.817236   
std        1.381587  140827.311974     12.574584      0.548138   
min       -1.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.423625     33.000000      1.000000   
50%        0.000000   -1203.369529     42.000000      1.000000   
75%        1.000000    -291.095954     53.000000      1.000000   
max       20.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt  total_income  
count      21525.000000  21525.000000  1.935100e+04  
mean           0.972544      0.080883  1.674223e+05  
std            1.420324      0.272661  1.029716e+05  
min            0.000000      0.000000  2.066726e+04  
25%            0.000000      0.000000  1.030532e+05  
50%            0.000000      0.000000  1.450179e+05  
75%            1.000000    

**Вывод по данным:**

Необходима предобработка. Названия столбцов однотипные, для удобной работы замены не требуют. 

Входные данные "сырые": 
* Часть значений в столбце 'days_employed' (для пенсионеров) вероятно имеет значения трудовой стаж в днях*24, а остальные данные имеют отрицательные значения.Трудовой стаж удобнее анализировать в годах.
* В столбце 'children' есть значения "-1" (вероятно ошибка знака) и значения "20" (вероятно ошибка порядка).
* Значение XNA в 'gender' встречается 1 раз.
* Есть малочисленные группы в 'income_type' (6 значений).
* В 'education' разный регистр.
* Есть пропуски ('days_employed', 'total_income').
* Есть ошибочные нулевые значения ('dob_years').

Данные о трудовом стаже собраны в новый столбец 'years_employed' (трудовой стаж в годах), столбец 'days_employed' удалил. . Удалил 4 малочисленные группы из 'income_type'.

<a name="3"></a>
## Предобработка данных

<a name="31"></a>
### Устранение ошибок

In [5]:
#функция для приведения стажа к единому виду
def days_years_employed (days_employed):
    '''
    принимает значение стажа и возвращает в годах
    если значение больше 36500 дней то считаем что оно указано в часах иначе в днях
    ''' 
    if days_employed>36500:
        return (days_employed)/(365*24)
    return abs(days_employed/(365))

In [6]:
#удаляю строки с 0 возрастом (меньше 0,5%)
data = data[data['dob_years']!=0] 

#удаляю малочисленные группы 'income_type'
data = data[(data['income_type']!='предприниматель') & (data['income_type']!='безработный') & (
    data['income_type']!='студент') & (data['income_type']!='в декрете')]

#добавляю столбец со стажем в годах
data['years_employed'] = data['days_employed'].apply(days_years_employed)

#меняю значение 'children' с -1 на 1 и 20 на 2
data['children'] = abs(data['children'])
data['children'] = data['children'].replace(20, 2)

#удаляю единичное значение 'gender' XNA
data=data[data['gender']!='XNA'] 

#удаляю столбцы 'days_employed', 'education_id', 'family_status_id'
data.drop(['days_employed', 'education_id', 'family_status_id'], axis='columns', inplace=True)

#привожу данные в столбце 'education' к нижнему регистру
data['education']=data['education'].str.lower()

<a name="32"></a>
### Обработка пропусков

In [7]:
nan_years_employed = data[data['years_employed'].isna()==True] #пропуски в years_employed
nan_total_income = data[data['total_income'].isna()==True] #пропуски в total_income
print('NaN в {:.0%} строк'.format(len(nan_years_employed)/len(data))) 

NaN в 10% строк


In [8]:
def median_groupby(row):
    '''
    функция для заполнения пропусков в столбце медианами по группе 
    
    получает строку, выдергивает значение столбца группировки (group_colomn) 
    возвращает значение соответствующей ему медианы
    
    До использования необходимо определить:
    nan_col - переменная задает столбец с пропуском
    group_col - переменная задает столбец с признаком по которому будем группировать для поиска медианы
    median_series - series со значениями медиан по группе
    '''
    try:
        group_col_value = row[group_col]
        median_value = median_series[group_col_value]
        return median_value
    except:
        return 'error median_value'

#блок обрабатывает пропуски в 'total_income' и 'years_employed'
    
#Заполняю пропуски в доходах ('total_income') медианой соответствующего типа занятости ('income_type')
nan_col = 'total_income' #столбец с пропуском
group_col = 'income_type'   #столбец по которому будут сгруппированы данные для поиска медианы
median_series = data.groupby(group_col)[nan_col].median() #series со значениями медиан по группе
data[nan_col] = data[nan_col].fillna(data.apply(median_groupby, axis=1))  

#Заполняю пропуски в стаже 'years_employed' медианой соответствующего возраста
nan_col = 'years_employed' #столбец с пропуском
group_col = 'dob_years'    #столбец по которому будем группировать для поиска медианы
median_series = data.groupby(group_col)[nan_col].median() #series со значениями медиан по группе
data[nan_col] = data[nan_col].fillna(data.apply(median_groupby, axis=1))

<a name="33"></a>
### Замена типа данных

In [9]:
#меняю столбец 'years_employed' и 'total_income' на целочисленный
data['years_employed'] = data['years_employed'].astype('int64') 
data['total_income'] = data['total_income'].astype('int64')

<a name="34"></a>
### Обработка дубликатов

In [10]:
data = data.drop_duplicates().reset_index(drop=True) #удаляю дубликаты

**Вывод:**

Данные в столбце 'education' привел к нижнему регистру. 

Строк с NaN - 10% (возможно данные из другого источника или сбой). Пропущенные значения трудового стажа заполнил медианой соответствующей возрастной группы, значения дохода - медианой соответствующего типа занятости (заполнение медианой соответствующего уровня образования на конечные результаты влияет незначительно).

Удалил дубликаты. Частично дубликаты связаны с наличием одинаковых строк с различными вариантами написания в столбце 'education', природа остальных не ясна. Т.к. в данных нет уникальных id заемщиков то среди удаленных дубликатов могли попасться совпадающие для разных людей значения, но такая вероятность крайне мала.

<a name="4"></a>
## Лемматизация целей кредита

In [11]:
#лемматизирую, "склеиваю", отбрасываю \n
m=Mystem()

data['purpose_lemm']=data['purpose'].apply(m.lemmatize).str.join('').str.strip()

print(data['purpose_lemm'].value_counts()) #4 цели: автомобиль, свадьба, недвижимость / жилье, образование

автомобиль                                965
свадьба                                   786
на проведение свадьба                     763
сыграть свадьба                           760
операция с недвижимость                   672
покупка коммерческий недвижимость         658
операция с коммерческий недвижимость      648
покупка жилье для сдача                   648
операция с жилье                          646
покупка жилье                             640
жилье                                     640
покупка жилье для семья                   637
строительство собственный недвижимость    632
недвижимость                              629
операция со свой недвижимость             627
строительство жилой недвижимость          621
покупка свой жилье                        619
строительство недвижимость                619
покупка недвижимость                      617
ремонт жилье                              604
покупка жилой недвижимость                602
на покупка свой автомобиль        

**Вывод**
Цели можно разделить на 4 категории: автомобиль, свадьба, недвижимость / жилье, образование

<a name="5"></a>
## Категоризация данных

In [12]:
#Функция возвращает категорию цели кредита
def purpose_groups (row):
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'свадьба' in row:
        return 'свадьба'
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'жилье' in row:
        return 'недвижимость'
    if 'образование' in row:
        return 'образование'
    return 'УТОЧНИТЬ'

data['purpose_group'] = data['purpose_lemm'].apply(purpose_groups)
data.drop(['purpose', 'purpose_lemm'], axis='columns', inplace=True) 
print(data['purpose_group'].value_counts())

недвижимость    10759
автомобиль       4283
образование      3995
свадьба          2309
Name: purpose_group, dtype: int64


In [13]:
#определяю 25, 50, 75 процентили 'total_income'
q25 = data['total_income'].quantile(0.25)
q50 = data['total_income'].quantile(0.50)
q75 = data['total_income'].quantile(0.75)
print(q25, q50, q75)

107663.25 142594.0 195751.75


In [14]:
#Функция возвращает категорию дохода
def total_income_groups (x):
    try:
        if x>200000:
            return 'более 200 тыс./мес.'
        if x>150000:
            return '150-200 тыс./мес.'
        if x>100000:
            return '100-150 тыс./мес.'
        if x<=100000:
            return 'менее 100 тыс./мес.'
    except:
        return 'ERROR'

#добавляем столбец с уровнем дохода
data['income_group']=data['total_income'].apply(total_income_groups)

**Вывод**
* Добавил столбец 'purpose_group' - категория цели кредита. 
* Разбил заемщиков на 4 категории по уровню дохода (в абсолютных величинах), добавил категорию в столбец 'income_group'.

<a name="6"></a>
## Ответы на вопросы задачи

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
#средний % должников
debtall=data['debt'].sum()/data['debt'].count()
print('% должников по всему датасету: {:.2%}'.format(debtall))

#% должников среди бездетных
debtchildfree=data[data['children']==0]['debt'].sum()/data[data['children']==0]['debt'].count()
print('% должников среди бездетных: {:.2%}'.format(debtchildfree))

#средний % должников среди заемщиков с детьми
debtchild=data[data['children']!=0]['debt'].sum()/data[data['children']!=0]['debt'].count()
print('% должников среди заемщиков с детьми: {:.2%}'.format(debtchild))
print()

#% должников в зависимости от количества детей
print('% должников в зависимости от количества детей:')
childrendebt=data.groupby('children')['debt'].agg(['sum', 'count'])
childrendebt['debt%']=childrendebt['sum']/childrendebt['count']*100
print(childrendebt)


% должников по всему датасету: 8.11%
% должников среди бездетных: 7.55%
% должников среди заемщиков с детьми: 9.18%

% должников в зависимости от количества детей:
           sum  count     debt%
children                       
0         1058  14017  7.547977
1          441   4838  9.115337
2          201   2113  9.512541
3           27    328  8.231707
4            4     41  9.756098
5            0      9  0.000000


**Вывод:**
Наличие детей (вне зависимости от их количества) увеличивает вероятность возникновения задолженности по кредиту. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [16]:
#% должников в зависимости от семейного положения
print('% должников в зависимости от семейного положения:')
familydebt=data.groupby('family_status')['debt'].agg(['sum', 'count'])
familydebt['debt%']=familydebt['sum']/familydebt['count']*100
print(familydebt.sort_values(by='debt%'))

% должников в зависимости от семейного положения:
                       sum  count     debt%
family_status                              
вдовец / вдова          62    954  6.498952
в разводе               85   1185  7.172996
женат / замужем        925  12287  7.528282
гражданский брак       386   4127  9.353041
Не женат / не замужем  273   2793  9.774436


**Вывод:**
Наиболее добросовестные заемщики - бывшие ранее в браке и находящиеся сейчас в официальном браке. Риск невозврата кредита для холостых, а также заемщиков с неоформленными супружескими отношениями возрастает.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [17]:
#% должников в зависимости от уровня дохода
print('% должников в зависимости от уровня дохода:')
incomedebt=data.groupby('income_group')['debt'].agg(['sum', 'count'])
incomedebt['debt%']=incomedebt['sum']/incomedebt['count']*100
print(incomedebt.sort_values(by='debt%'))

% должников в зависимости от уровня дохода:
                     sum  count     debt%
income_group                             
более 200 тыс./мес.  357   5037  7.087552
менее 100 тыс./мес.  351   4437  7.910751
150-200 тыс./мес.    402   4742  8.477436
100-150 тыс./мес.    621   7130  8.709677


**Вывод:** 
Наиболее дисциплинированы заемщики с доходом свыше 200 тыс./мес., а также категория с доходом менее 100 тыс./мес (возможно связано с меньшим размером кредита). Риск невозврата для средних категорий возрастает.

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
#% должников в зависимости от цели кредита
print('% должников в зависимости от цели кредита:')
purposedebt=data.groupby('purpose_group')['debt'].agg(['sum', 'count'])
purposedebt['debt%']=purposedebt['sum']/purposedebt['count']*100
print(purposedebt.sort_values(by='debt%'))

% должников в зависимости от цели кредита:
               sum  count     debt%
purpose_group                      
недвижимость   778  10759  7.231155
свадьба        184   2309  7.968818
образование    370   3995  9.261577
автомобиль     399   4283  9.315900


**Вывод:**
Наименьший риск в категории кредитов на недвижимость, наибольший у автокредитов и кредитов на образование.

In [19]:
data_pivot=data.pivot_table(index = ['family_status'], columns= 'gender', values= 'debt', aggfunc=['count', 'sum', 'mean'])
display(data_pivot)
print()
data_pivot2=data.pivot_table(index = ['income_type'], values= 'debt', aggfunc=['count', 'sum', 'mean'])
display(data_pivot2)

count        sum           mean          
gender                    F     M    F    M         F         M
family_status                                                  
Не женат / не замужем  1721  1072  118  155  0.068565  0.144590
в разводе               927   258   61   24  0.065804  0.093023
вдовец / вдова          899    55   51   11  0.056730  0.200000
гражданский брак       2827  1300  232  154  0.082066  0.118462
женат / замужем        7725  4562  526  399  0.068091  0.087462

,count,sum,mean
,debt,debt,debt
income_type,,,
госслужащий,1451,86,0.059269
компаньон,5057,375,0.074155
пенсионер,3809,215,0.056445
сотрудник,11029,1055,0.095657


**Вывод:**
Риски по группам с различным семейным положением значительно отличаются в зависимости от пола заемщика. Это необходимо учитывать.

<a name="7"></a>
## Общий вывод

Исследование показало что статистика о платежеспособности клиентов может быть использована для построения модели кредитного скоринга. 
_____
Информация о заемщике влияет на риски в следующем по значимости порядке: 

**семейное положение**  
Наиболее добросовестные заемщики - бывшие ранее в браке и находящиеся сейчас в официальном браке. Риск невозврата кредита для холостых, а также заемщиков с неоформленными супружескими отношениями возрастает.

**наличие детей**  
Наличие детей (вне зависимости от их количества) увеличивает вероятность возникновения задолженности по кредиту.

**цель получения кредита**  
Наименьший риск в категории кредитов на недвижимость, наибольший у автокредитов и кредитов на образование.

**уровень дохода**  
Наиболее дисциплинированы заемщики с доходом свыше 200 тыс./мес., а также категория с доходом менее 100 тыс./мес (возможно связано с меньшим размером кредита). Риск невозврата для средних категорий возрастает.

Также необходимо принимать во внимание прочую информацию о клиенте (к примеру статистика по группам значительно отличается для мужчин и для женщин).

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.